In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os

import sys
from importlib import reload
from sklearn.metrics import confusion_matrix
import tensorflow as tf
#import keras
from scipy import stats
import scipy.signal as scisig
import scipy.stats
import dalmatian
from sklearn.metrics import f1_score, precision_score, recall_score
import seaborn as sns
#from tqdm.keras import TqdmCallback

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import np_utils
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score

from keras.layers import LSTM, Dense, Conv1D, TimeDistributed, Flatten, Activation, Dropout, Bidirectional
from keras.callbacks import History, TensorBoard, Callback
import keras.initializers as KI
from keras.layers import BatchNormalization

from sklearn.preprocessing import StandardScaler
import scipy.stats as stats
import tensorflow_addons as tfa
from tqdm.keras import TqdmCallback

In [2]:
def f(i):
    return pd.read_csv(i, index_col=0)

data_path = "../../data/output/E4_mapped/"
output_path = "../../data/output/E4_comb_mods/"
participants = ["p" + str(i).zfill(2) for i in range(1, 16) if i != 3 and i != 4]

def participant_data(data):
    
    for p in participants:
        files = [f for f in sorted(glob.glob(data_path + str(p) + "/*" + str(data) + ".csv"))]
        p_data = pd.concat(map(f, files), ignore_index=True)
        #p_data.to_csv(output_path + str(p) + "_" + str(data) + '.csv')

In [22]:
modalities = ["ACC", "EDA", "TEMP", "BVP", "HR"]
for m in modalities:
    participant_data(m)

In [32]:
conf_matrix_list_of_arrays = []
scores = []
acc_per_fold = []
loss_per_fold = []
f1_per_fold = []
prec_per_fold = []
rec_per_fold = []

def s(samples):
    std = np.std(samples)
    if std == 0:
        return samples - np.mean(samples)
    else:
        return (samples - np.mean(samples)) / std

acc_freq = 32
eda_freq = 4
temp_freq = 4
bvp_freq = 64
hr_freq = 1

#sliding window - set window size and step size 
def create_window(pid, win_size=30, win_step=15, Binary=False):
    
    for f in os.listdir(output_path):
        if f.startswith(str(pid)) and f.endswith("ACC.csv"):
            acc_path = f
        elif f.startswith(str(pid)) and f.endswith("EDA.csv"):
            eda_path = f
        elif f.startswith(str(pid)) and f.endswith("TEMP.csv"):
            temp_path = f
        elif f.startswith(str(pid)) and f.endswith("BVP.csv"):
            bvp_path = f
        elif f.startswith(str(pid)) and f.endswith("HR.csv"):
            hr_path = f
    
    acc_data = pd.read_csv(output_path + acc_path, index_col=0 )
    eda_data = pd.read_csv(output_path + eda_path, index_col=0)
    temp_data = pd.read_csv(output_path + temp_path, index_col=0)
    bvp_data = pd.read_csv(output_path + bvp_path, index_col=0)
    #hr_data = pd.read_csv(output_path + hr_path, index_col=0)

    eda_data['interest'] = eda_data['interest'].replace([1,2,3,4], [0,1,2,3])
    bvp_data['understand'] = bvp_data['understand'].replace([1,2,3,4], [0,1,2,3]) 
    
    acc_x = []; acc_y = []; acc_z = []; eda = []; temp=[]; label=[]; bvp=[]; hr=[]; ut=[]; win_size = 30 ; win_step= 15; 
    total_time = int(len(eda_data) / eda_freq)

    for i in range(win_size, total_time, win_step):

        acc_x.append(s(acc_data["acc_x"][acc_freq * (i - win_size): acc_freq * i]))
        acc_y.append(s(acc_data["acc_y"][acc_freq * (i - win_size): acc_freq * i]))
        acc_z.append(s(acc_data["acc_z"][acc_freq * (i - win_size): acc_freq * i]))

        eda.append(s(eda_data["data"][eda_freq * (i - win_size): eda_freq * i]))
        temp.append(s(temp_data["data"][temp_freq * (i - win_size): temp_freq * i]))
        bvp.append(s(bvp_data["data"][bvp_freq * (i - win_size): bvp_freq * i]))
        #hr.append(s(hr_data["data"][hr_freq * (i - win_size): hr_freq * i]))
        ut.append(bvp_data["understand"][bvp_freq * (i - win_size): bvp_freq * i])

        l = stats.mode(eda_data["interest"][eda_freq * (i - win_size): eda_freq * i])[0][0]
        if Binary:
            if(l == 0) | (l == 1):
                l = 0
                label.append(l)
            else:
                l = 1
                label.append(l)
        else:

        #l_vote = np.bincount(np.array(l)).argmax()
            label.append(l)


    return {
        "label": np.array(label),
        "acc_x": np.array(acc_x), 
        "acc_y": np.array(acc_y), 
        "acc_z": np.array(acc_z),
        "eda": np.array(eda),
         "bvp": np.array(bvp), 
         "temp": np.array(temp),
         "ut": np.array(ut)}

In [33]:
def train(train_dataset, test_dataset):
    models = []
    for feature in ["acc_x", "acc_y", "acc_z", "eda", "bvp", "temp" ,"ut"]:
    #for feature in [ "bvp", "eda", "temp", "ut"]:

        input_size = train_dataset[feature].shape[1]
        #filter_size = int(input_size / 10)  # hyper param
        #filter_size = 3
        input_layer = tf.keras.layers.Input(shape=(input_size, 1))
        conv_layer_1 = tf.keras.layers.Conv1D(
            50, 3, activation="relu",
            kernel_regularizer=tf.keras.regularizers.l2(1e-3))(input_layer)
        conv_layer_2 = tf.keras.layers.Conv1D(
            30, 3, activation="relu",
            kernel_regularizer=tf.keras.regularizers.l2(1e-3))(conv_layer_1)
        conv_layer_3 = tf.keras.layers.Conv1D(
            10, 3, activation="relu",
            kernel_regularizer=tf.keras.regularizers.l2(1e-3))(conv_layer_2)
        pooling_layer = tf.keras.layers.GlobalAveragePooling1D()(conv_layer_3)
        models.append(tf.keras.Model(inputs=input_layer, outputs=pooling_layer))

    concat_layer = tf.keras.layers.concatenate([x.output for x in models])
    dense_layer_1 = tf.keras.layers.Dense(128, activation="relu")(concat_layer)
    dense_layer_2 = tf.keras.layers.Dense(64, activation="relu")(concat_layer)
    output_layer = tf.keras.layers.Dense(1, activation="sigmoid")(dense_layer_2)

    model = tf.keras.Model(inputs=[x.input for x in models], outputs=output_layer)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss="binary_crossentropy", metrics=['accuracy'])

    X_train = []; scalers = {}
    for feature in ["acc_x", "acc_y", "acc_z", "eda", "bvp", "temp" ,"ut"]:
    #for feature in [ "bvp", "eda", "temp", "ut"]:

        # scalers[feature] = StandardScaler()
        # scalers[feature].fit(train_dataset[feature])
        # scaled = scalers[feature].transform(train_dataset[feature])
        # X_train.append(np.reshape(scaled, (-1, scaled.shape[1], 1)))
        X_train.append(np.reshape(train_dataset[feature], (-1, train_dataset[feature].shape[1], 1)))
    y_train = train_dataset["label"]

    X_test = []
    for feature in ["acc_x", "acc_y", "acc_z", "eda", "bvp", "temp" ,"ut"]:
    #for feature in [ "bvp", "eda", "temp", "ut"]:

        # scaled = scalers[feature].transform(test_dataset[feature])
        # X_test.append(np.reshape(scaled, (-1, scaled.shape[1], 1)))
        X_test.append(np.reshape(test_dataset[feature], (-1, test_dataset[feature].shape[1], 1)))
    y_test = test_dataset["label"]


    history = model.fit(X_train, y_train, epochs=10, batch_size=512, validation_split=0.2, verbose=1)
    scores = model.evaluate(X_test, y_test)
    print(f'Score : {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1])
    loss_per_fold.append(scores[0])
    
    
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    f1_per_fold.append(f1_score(y_test, y_pred, average='weighted'))
    prec_per_fold.append(precision_score(y_test, y_pred, average='weighted'))
    rec_per_fold.append(recall_score(y_test, y_pred, average='weighted'))
    print('F1 score : ', f1_score(y_test, y_pred, average='weighted'))

    # pd.DataFrame(history.history).plot(figsize=(8, 5))
    # plt.show()

    cm = confusion_matrix(y_test, y_pred, labels=[0,1])
    print('Confusion Matrix : ')
    print(cm)
    
    conf_matrix_list_of_arrays.append(cm)
    return conf_matrix_list_of_arrays, acc_per_fold, f1_per_fold


In [34]:
comb_data = [create_window(x, Binary=True) for x in participants]

In [35]:
for test_p in range(len(comb_data)):
    train_data = {}
    train_data["acc_x"] = np.concatenate([comb_data[i]["acc_x"] for i in range(len(comb_data)) if i != test_p])
    train_data["acc_y"] = np.concatenate([comb_data[i]["acc_y"] for i in range(len(comb_data)) if i != test_p])
    train_data["acc_z"] = np.concatenate([comb_data[i]["acc_z"] for i in range(len(comb_data)) if i != test_p])
    train_data["eda"] = np.concatenate([comb_data[i]["eda"] for i in range(len(comb_data)) if i != test_p])
    train_data["bvp"] = np.concatenate([comb_data[i]["bvp"] for i in range(len(comb_data)) if i != test_p])
    train_data["temp"] = np.concatenate([comb_data[i]["temp"] for i in range(len(comb_data)) if i != test_p])
    train_data["ut"] = np.concatenate([comb_data[i]["ut"] for i in range(len(comb_data)) if i != test_p])

    train_data["label"] = np.concatenate([comb_data[i]["label"] for i in range(len(comb_data)) if i != test_p])

    test_data = comb_data[test_p]
    cmf, acc, f1 = train(train_data, test_data)

Epoch 1/10
5/5 [==============================] - 11s 2s/step - loss: 1.0724 - accuracy: 0.5343 - val_loss: 1.0649 - val_accuracy: 0.7113
Epoch 2/10
5/5 [==============================] - 8s 2s/step - loss: 1.0659 - accuracy: 0.6232 - val_loss: 1.0579 - val_accuracy: 0.6476
Epoch 3/10
5/5 [==============================] - 8s 2s/step - loss: 1.0602 - accuracy: 0.5950 - val_loss: 1.0514 - val_accuracy: 0.6166
Epoch 4/10
5/5 [==============================] - 8s 2s/step - loss: 1.0548 - accuracy: 0.5710 - val_loss: 1.0451 - val_accuracy: 0.6134
Epoch 5/10
5/5 [==============================] - 8s 2s/step - loss: 1.0494 - accuracy: 0.5575 - val_loss: 1.0390 - val_accuracy: 0.6134
Epoch 6/10
5/5 [==============================] - 8s 2s/step - loss: 1.0438 - accuracy: 0.5555 - val_loss: 1.0330 - val_accuracy: 0.6134
Epoch 7/10
5/5 [==============================] - 8s 2s/step - loss: 1.0382 - accuracy: 0.5591 - val_loss: 1.0270 - val_accuracy: 0.6134
Epoch 8/10
5/5 [========================

/Users/jai/opt/anaconda3/envs/dl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
5/5 [==============================] - 10s 2s/step - loss: 1.0902 - accuracy: 0.4696 - val_loss: 1.0957 - val_accuracy: 0.3921
Epoch 2/10
5/5 [==============================] - 8s 1s/step - loss: 1.0814 - accuracy: 0.4696 - val_loss: 1.0848 - val_accuracy: 0.3921
Epoch 3/10
5/5 [==============================] - 8s 2s/step - loss: 1.0735 - accuracy: 0.4696 - val_loss: 1.0748 - val_accuracy: 0.3921
Epoch 4/10
5/5 [==============================] - 8s 2s/step - loss: 1.0661 - accuracy: 0.4760 - val_loss: 1.0653 - val_accuracy: 0.4161
Epoch 5/10
5/5 [==============================] - 8s 2s/step - loss: 1.0589 - accuracy: 0.6565 - val_loss: 1.0572 - val_accuracy: 0.7654
Epoch 6/10
5/5 [==============================] - 8s 2s/step - loss: 1.0524 - accuracy: 0.7238 - val_loss: 1.0497 - val_accuracy: 0.6969
Epoch 7/10
5/5 [==============================] - 8s 2s/step - loss: 1.0466 - accuracy: 0.6878 - val_loss: 1.0433 - val_accuracy: 0.6764
Epoch 8/10
5/5 [========================

/Users/jai/opt/anaconda3/envs/dl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


F1 score :  0.12984994247386844
Confusion Matrix : 
[[103   0]
 [253   0]]
Epoch 1/10
5/5 [==============================] - 12s 2s/step - loss: 1.0780 - accuracy: 0.3583 - val_loss: 1.0748 - val_accuracy: 0.2781
Epoch 2/10
5/5 [==============================] - 8s 2s/step - loss: 1.0687 - accuracy: 0.5346 - val_loss: 1.0652 - val_accuracy: 0.6077
Epoch 3/10
5/5 [==============================] - 9s 2s/step - loss: 1.0606 - accuracy: 0.6832 - val_loss: 1.0571 - val_accuracy: 0.6222
Epoch 4/10
5/5 [==============================] - 8s 2s/step - loss: 1.0533 - accuracy: 0.6860 - val_loss: 1.0496 - val_accuracy: 0.6158
Epoch 5/10
5/5 [==============================] - 9s 2s/step - loss: 1.0463 - accuracy: 0.6896 - val_loss: 1.0419 - val_accuracy: 0.6190
Epoch 6/10
5/5 [==============================] - 9s 2s/step - loss: 1.0392 - accuracy: 0.6989 - val_loss: 1.0346 - val_accuracy: 0.6222
Epoch 7/10
5/5 [==============================] - 8s 2s/step - loss: 1.0325 - accuracy: 0.7049 - val_l

...

/Users/jai/opt/anaconda3/envs/dl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
5/5 [==============================] - 10s 2s/step - loss: 1.0491 - accuracy: 0.6400 - val_loss: 1.0704 - val_accuracy: 0.4104
Epoch 2/10
5/5 [==============================] - 8s 2s/step - loss: 1.0396 - accuracy: 0.6563 - val_loss: 1.0701 - val_accuracy: 0.4104
Epoch 3/10
5/5 [==============================] - 8s 2s/step - loss: 1.0305 - accuracy: 0.6790 - val_loss: 1.0695 - val_accuracy: 0.4104
Epoch 4/10
5/5 [==============================] - 8s 2s/step - loss: 1.0220 - accuracy: 0.6911 - val_loss: 1.0687 - val_accuracy: 0.4104
Epoch 5/10
5/5 [==============================] - 8s 2s/step - loss: 1.0135 - accuracy: 0.7045 - val_loss: 1.0671 - val_accuracy: 0.4104
Epoch 6/10
5/5 [==============================] - 8s 2s/step - loss: 1.0049 - accuracy: 0.7108 - val_loss: 1.0652 - val_accuracy: 0.4104
Epoch 7/10
5/5 [==============================] - 8s 2s/step - loss: 0.9963 - accuracy: 0.7125 - val_loss: 1.0631 - val_accuracy: 0.4104
Epoch 8/10
5/5 [========================

/Users/jai/opt/anaconda3/envs/dl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
5/5 [==============================] - 11s 2s/step - loss: 1.0392 - accuracy: 0.5840 - val_loss: 1.0498 - val_accuracy: 0.5210
Epoch 2/10
5/5 [==============================] - 8s 2s/step - loss: 1.0275 - accuracy: 0.5832 - val_loss: 1.0454 - val_accuracy: 0.5210
Epoch 3/10
5/5 [==============================] - 8s 2s/step - loss: 1.0176 - accuracy: 0.5832 - val_loss: 1.0410 - val_accuracy: 0.5210
Epoch 4/10
5/5 [==============================] - 8s 2s/step - loss: 1.0080 - accuracy: 0.5828 - val_loss: 1.0367 - val_accuracy: 0.5210
Epoch 5/10
5/5 [==============================] - 8s 2s/step - loss: 0.9995 - accuracy: 0.5844 - val_loss: 1.0324 - val_accuracy: 0.5210
Epoch 6/10
5/5 [==============================] - 8s 2s/step - loss: 0.9909 - accuracy: 0.5905 - val_loss: 1.0280 - val_accuracy: 0.5210
Epoch 7/10
5/5 [==============================] - 8s 2s/step - loss: 0.9823 - accuracy: 0.6181 - val_loss: 1.0235 - val_accuracy: 0.5210
Epoch 8/10
5/5 [========================

/Users/jai/opt/anaconda3/envs/dl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [38]:
np.mean(acc)

0.6081111958393683

In [37]:
acc

[0.7971014380455017,
 0.8483145833015442,
 0.6686747074127197,
 0.8872548937797546,
 0.3253588378429413,
 0.5866666436195374,
 0.18465909361839294,
 0.6000000238418579,
 0.942148745059967,
 0.30399999022483826,
 0.6047903895378113,
 0.7024221420288086,
 0.454054057598114]

In [24]:
acc

[0.7971014380455017,
 0.7106741666793823,
 0.6566265225410461,
 0.7745097875595093,
 0.3253588378429413,
 0.5866666436195374,
 0.18465909361839294,
 0.5593220591545105,
 0.942148745059967,
 0.30399999022483826,
 0.6047903895378113,
 0.7024221420288086,
 0.454054057598114]